In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from pulp import *
%matplotlib inline




Bad key "text.kerning_factor" on line 4 in
C:\Users\zacha\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


# Read in Data, Clean and Aggregate

In [2]:

#choose week to Optmize Roster
week=10

fd_string=f"Resources/Week_{week}/FD_Player_List_Week_{week}.csv"
rb_string=f"Resources/Week_{week}/FantasyPros_2020_Week_{week}_RB_Rankings.csv"
qb_string=f"Resources/Week_{week}/FantasyPros_2020_Week_{week}_QB_Rankings.csv"
wr_string=f"Resources/Week_{week}/FantasyPros_2020_Week_{week}_WR_Rankings.csv"
te_string=f"Resources/Week_{week}/FantasyPros_2020_Week_{week}_TE_Rankings.csv"
d_string=f"Resources/Week_{week}/FantasyPros_2020_Week_{week}_DST_Rankings.csv"

In [3]:

fd_path=Path(fd_string)
rb_path=Path(rb_string)
qb_path=Path(qb_string)
wr_path=Path(wr_string)
te_path=Path(te_string)
d_path=Path(d_string)

players=pd.read_csv(fd_path, index_col="Nickname")
rb=pd.read_csv(rb_path,index_col="PLAYER NAME")
qb=pd.read_csv(qb_path,index_col="PLAYER NAME")
wr=pd.read_csv(wr_path,index_col="PLAYER NAME")
te=pd.read_csv(te_path,index_col="PLAYER NAME")
d=pd.read_csv(d_path,index_col="PLAYER NAME")

fp_players=pd.concat([rb, qb,wr,te,d],axis="rows",join="inner")
#players["Proj"]=""
#all_data=pd.merge(players,fp_players["PROJ. FPTS"])
#all_data.head()
proj=pd.DataFrame(fp_players["PROJ. FPTS"])
players=players.join(proj)
players["Nickname"]=players.index
#players["PROJ. FPTS"]
#players=players.pd.Index.set_names(names="Nickname")
players.head()


,Id,Position,First Name,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,PROJ. FPTS,Nickname
A.J. Green,51566-14187,WR,A.J.,Green,5.887500,8,5600,CIN@PIT,CIN,PIT,NaN,NaN,NaN,9.3,A.J. Green
A.J. Richardson,51566-54186,WR,A.J.,Richardson,0.000000,2,4500,BUF@ARI,ARI,BUF,NaN,NaN,NaN,NaN,A.J. Richardson
AJ Dillon,51566-93107,RB,AJ,Dillon,1.966667,6,4900,JAC@GB,GB,JAC,IR,Illness,NaN,1.5,AJ Dillon
AJ McCarron,51566-31047,QB,AJ,McCarron,8.950000,2,6000,HOU@CLE,HOU,CLE,NaN,NaN,NaN,NaN,AJ McCarron
Aaron Fuller,51566-71756,WR,Aaron,Fuller,0.000000,1,4500,SEA@LAR,SEA,LAR,NaN,NaN,NaN,NaN,Aaron Fuller


In [4]:
players_filtered=players[players["Injury Indicator"].isnull()]
players_filtered=players_filtered[players_filtered["PROJ. FPTS"].notnull()]
#players_filtered.dropna(axis=0,how="all",subset=["PROJ. FPTS"])
players_filtered.head()

,Id,Position,First Name,Last Name,FPPG,Played,Salary,Game,Team,Opponent,Injury Indicator,Injury Details,Tier,PROJ. FPTS,Nickname
A.J. Green,51566-14187,WR,A.J.,Green,5.887500,8,5600,CIN@PIT,CIN,PIT,NaN,NaN,NaN,9.3,A.J. Green
Aaron Jones,51566-40442,RB,Aaron,Jones,19.400000,6,8800,JAC@GB,GB,JAC,NaN,NaN,NaN,19.4,Aaron Jones
Aaron Rodgers,51566-6894,QB,Aaron,Rodgers,23.514999,8,8400,JAC@GB,GB,JAC,NaN,NaN,NaN,22.7,Aaron Rodgers
Adam Shaheen,51566-79967,TE,Adam,Shaheen,3.383333,6,4200,LAC@MIA,MIA,LAC,NaN,NaN,NaN,2.8,Adam Shaheen
Adam Trautman,51566-130270,TE,Adam,Trautman,4.075000,4,4200,SF@NO,NO,SF,NaN,NaN,NaN,2.7,Adam Trautman


In [5]:
#Choose whether to use FPPG or Projected Points
pt="PROJ. FPTS"

play=players_filtered[["Position","Nickname","Salary",pt]]

play.head()

,Position,Nickname,Salary,PROJ. FPTS
A.J. Green,WR,A.J. Green,5600,9.3
Aaron Jones,RB,Aaron Jones,8800,19.4
Aaron Rodgers,QB,Aaron Rodgers,8400,22.7
Adam Shaheen,TE,Adam Shaheen,4200,2.8
Adam Trautman,TE,Adam Trautman,4200,2.7


In [6]:
#update the df to clean "-" points from dataframe
play.loc[(play[pt]=="-"),pt]=0

play[play[pt]=="-"]
play[[pt]]=play[[pt]].astype(float)

play.dtypes

C:\Users\zacha\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\zacha\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


Position       object
Nickname       object
Salary          int64
PROJ. FPTS    float64
dtype: object

# Run Optimization Problem

In [7]:
#Initialize budget constraint
budget=60000

#initialize dictionary for salary and points data
salaries={}
points={}

#split players up into seperate dataframes by 
for pos in play["Position"].unique():
    available_pos=play[play["Position"]==pos]
    
    #create a dictionary of player nicknames and salaries
    salary=list(available_pos[["Nickname","Salary"]].set_index("Nickname").to_dict().values())[0]
    
    #create a dictionary of player points
    point=list(available_pos[["Nickname",pt]].set_index("Nickname").to_dict().values())[0]
    salaries[pos]=salary
    points[pos]=point

#maximum number of players you can have at each position
pos_constraints_max={
    "QB":1,
    "RB":3,
    "WR":4,
    "TE":2,
    "FLEX":1,
    "D": 1
    
}


#minimum number of players you can have at each position
pos_constraints_min={
    "QB":1,
    "RB":2,
    "WR":3,
    "TE":1,
    "FLEX":1,
    "D": 1
    
}

#define what postions can serve as flexes
pos_flex={
     "QB":0,
    "RB":1,
    "WR":1,
    "TE":1,
    "FLEX":0,
    "D": 0
}

#limit the number of flex players
pos_flex_available= 5

In [8]:
#Make sure salary dictionary set up correctly
salaries["D"]

{'Arizona Cardinals': 3600,
 'Buffalo Bills': 4100,
 'Carolina Panthers': 3000,
 'Cincinnati Bengals': 3400,
 'Cleveland Browns': 4400,
 'Denver Broncos': 4300,
 'Detroit Lions': 3900,
 'Green Bay Packers': 4800,
 'Houston Texans': 3500,
 'Jacksonville Jaguars': 3100,
 'Las Vegas Raiders': 3500,
 'Los Angeles Chargers': 4100,
 'Los Angeles Rams': 4000,
 'Miami Dolphins': 4700,
 'New Orleans Saints': 4500,
 'New York Giants': 3900,
 'Philadelphia Eagles': 4600,
 'Pittsburgh Steelers': 5000,
 'San Francisco 49ers': 3700,
 'Seattle Seahawks': 3700,
 'Tampa Bay Buccaneers': 4200,
 'Washington Football Team': 3800}

In [9]:
budget=60000

_vars={k: LpVariable.dict(k,v,cat="Binary") for k, v in points.items()}

prob=LpProblem("Fantasy", LpMaximize)
position_constraints=[]
rewards=[]
costs=[]


#Setting up reward

for k, v in _vars.items():
    
    #update the points projection
    rewards+=lpSum([points[k][i]*_vars[k][i] for i in v])
   
    #update constraints
    costs+=lpSum([salaries[k][i]*_vars[k][i] for i in v])
    prob+=lpSum([_vars[k][i] for i in v])>=pos_constraints_min[k]
    prob+=lpSum([_vars[k][i] for i in v])<=pos_constraints_max[k]
    prob+=lpSum([pos_flex[k]*_vars[k][i] for i in v])<=pos_flex_available

    
prob+=lpSum(rewards)
prob+=lpSum(costs)<=budget
prob.solve()

1

In [10]:
#prob.constraints


In [11]:

def output(prob):
    div = '---------------------------------------\n'
    print("Optimal Roster:")
    print(div)
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name)
    print(div)
    print("Budget Constraint Overview:")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Projected Fantasy Points:")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))
    

output(prob)

Optimal Roster:
---------------------------------------

D_Washington_Football_Team
QB_Kyler_Murray
RB_Aaron_Jones
RB_Antonio_Gibson
RB_Mike_Davis
TE_Taysom_Hill
WR_Chase_Claypool
WR_Davante_Adams
WR_Terry_McLaurin
---------------------------------------

Budget Constraint Overview:
3800*1.0 + 8800*1.0 + 8800*1.0 + 6100*1.0 + 5400*1.0 + 4500*1.0 + 6100*1.0 + 9500*1.0 + 7000*1.0 = 60000.0
---------------------------------------

Projected Fantasy Points:
7.1*1.0 + 24.2*1.0 + 19.4*1.0 + 13.0*1.0 + 13.4*1.0 + 14.4*1.0 + 12.7*1.0 + 20.3*1.0 + 15.3*1.0 = 139.8
